In [ ]:
import os
import numpy as np
from keras.applications import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
g_dir = './mushroom/valAndTrain/train'

In [ ]:
def loadImages(data_dir):
   
    data = {}
    for c in ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']:
        data[c] = []
        if c == 'Agaricus':
            label = 0
        elif c == 'Amanita':
            label = 1
        elif c == 'Boletus':
            label = 2
        elif c == 'Cortinarius':
            label = 3
        elif c == 'Entoloma':
            label = 4
        elif c == 'Hygrocybe':
            label = 5
        elif c == 'Lactarius':
            label = 6
        elif c == 'Russula':
            label = 7
        elif c == 'Suillus':
            label = 8

        files = os.listdir(data_dir + '/' + c)
        files = files[0:1200] # Maximum number objects for each class

#         print("A sample object from {} class:".format(c))
#         plt.imshow(image.load_img(data_dir + '/' + c + '/' + files[4], target_size=(224, 224)))
#         plt.show()

        for f_name in files:
            img_path = data_dir + '/' + c + '/' + f_name
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            data[c].append([x, label])
#         print(data[c])
   
    return data

In [ ]:
data = loadImages(g_dir)
# print(data)

In [ ]:
train_classObj1, test_classObj1 = train_test_split(data['Agaricus'], test_size = 0.2)
train_classObj2, test_classObj2 = train_test_split(data['Amanita'], test_size = 0.2)
train_classObj3, test_classObj3 = train_test_split(data['Boletus'], test_size = 0.2)
train_classObj4, test_classObj4 = train_test_split(data['Cortinarius'], test_size = 0.2)
train_classObj5, test_classObj5 = train_test_split(data['Entoloma'], test_size = 0.2)
train_classObj6, test_classObj6 = train_test_split(data['Hygrocybe'], test_size = 0.2)
train_classObj7, test_classObj7 = train_test_split(data['Lactarius'], test_size = 0.2)
train_classObj8, test_classObj8 = train_test_split(data['Russula'], test_size = 0.2)
train_classObj9, test_classObj9 = train_test_split(data['Suillus'], test_size = 0.2)

print("The number of TRAIN objects in each class: ", len(train_classObj1))
print("The number of TEST objects in each class: ", len(test_classObj1))

train_objs = train_classObj1 + train_classObj2 + train_classObj3+train_classObj4 + train_classObj5 + train_classObj6+train_classObj7+ train_classObj8 + train_classObj9
test_objs = test_classObj1 + test_classObj2 + test_classObj3+test_classObj4 + test_classObj5 + test_classObj6+test_classObj7 + test_classObj8 + test_classObj9

In [ ]:
train_data = [x[0] for x in train_objs]
train_label = [y[1] for y in train_objs]

test_data = [x[0] for x in test_objs]
test_label = [y[1] for y in test_objs]

In [ ]:
X_train = np.array(train_data)
X_test = np.array(test_data)

In [ ]:
train_data = np.squeeze(X_train)
train_label = np.array(train_label)

In [ ]:
test_data = np.squeeze(X_test)
test_label = np.array(test_label)

In [ ]:
print("Input dimension: ", train_data.shape)

In [ ]:
base_model = Xception(weights='imagenet', include_top=False)

In [ ]:
training_feature = [base_model.predict(x).flatten() for x in X_train]
test_feature = [base_model.predict(x).flatten() for x in X_test]

training_feature = np.array(training_feature)
test_feature = np.array(test_feature)

In [ ]:
print("Training feature dimension: ", training_feature.shape)
print("Test feature dimension: ", test_feature.shape)

In [ ]:
# PCA
pca = PCA(n_components=2)
training_feature_pca_2D = pca.fit_transform(training_feature)
test_feature_pca_2D = pca.transform(test_feature)

In [ ]:
pca = PCA(n_components=3)
training_feature_pca_3D = pca.fit_transform(training_feature)
test_feature_pca_3D = pca.transform(test_feature)

In [ ]:
#pca 2d
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
colors = ['r', 'g', 'b','r', 'g', 'c','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']
for class_ix, marker, color, label in zip(classes, markers, colors, labels):
     ax.scatter(training_feature_pca_2D[np.where(train_label == class_ix), 0],
             training_feature_pca_2D[np.where(train_label == class_ix), 1],
             marker=marker, color=color,
             linewidth='1', alpha=0.9, label=label)  
#          ax.legend(loc='best')
plt.xlabel('First principal component')
plt.ylabel('Second Principal Component')
plt.title("2 component PCA")
plt.show()

In [ ]:
#pca 3d
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
colors = ['r', 'g', 'b','r', 'g', 'b','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']

ax.scatter(training_feature_pca_3D[:, 0],
             training_feature_pca_3D[:, 1],
             training_feature_pca_3D[:, 2],
             c=train_label)

ax.set_xlabel('First component PCA')
ax.set_ylabel('Second component PCA')
ax.set_zlabel('Third component PCA')
plt.title("3 component PCA")
plt.show()

In [ ]:
model = SVC(kernel ='rbf', gamma='scale', C = 1)

In [ ]:
# 2D Training
from numpy import array
# print（map(list, zip(*training_feature_pca_2D))）
print(array(training_feature_pca_2D))
# print(training_feature_pca_2D.tolist())
print(train_label)
print(np.array(train_label.transpose()))
model.fit(training_feature_pca_2D, float(train_label))

# Evaluation
y_pred = model.predict(test_feature_pca_2D)

# Result
acc = accuracy_score(test_label, y_pred)
print("Accuracy: {:.2f}".format(acc*100))